In [17]:
import pandas as pd
import numpy as np


In [19]:
training = pd.read_csv("mapping_with_label_training.csv").dropna()
training = training[training["form_type"] != "4"]
training = training[training["label"] != 0]
twoway = pd.crosstab(training["form_type"], 
training["label"], normalize="index")
twoway = twoway.merge(training.groupby("form_type")\
                      .agg({"file_number":"count"}),
                      left_index=True,right_index=True)
twoway['prediction'] = np.where(
    twoway[-1] >= twoway[1], -1, 1)
twoway = twoway.reset_index()
twoway.sort_values("file_number", ascending=False).to_csv("two_way_probablistic.csv")

In [14]:
test = pd.read_csv("mapping_with_label_test.csv", index_col=0).dropna()
# test["label"] = test["label"].replace(0, 1)
test = test[test["form_type"] != "4"]
test = test[test["label"] != 0]
test = test[["form_type", "label"]]
pred = twoway[["form_type", "prediction"]]
test = test.merge(pred, on=["form_type"])
test

,form_type,label,prediction
0,3,1.0,-1
1,3,-1.0,-1
2,3,-1.0,-1
3,3,1.0,-1
4,3,-1.0,-1
...,...,...,...
31960,25-NSE/A,1.0,-1
31961,NT 10-Q,1.0,-1
31962,PRER14A,1.0,-1
31963,PREM14A,-1.0,-1


In [15]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print(confusion_matrix(test["label"], test["prediction"]))
print(classification_report(test["label"], test["prediction"]))

[[ 2085 13121]
 [ 2421 14338]]
              precision    recall  f1-score   support

        -1.0       0.46      0.14      0.21     15206
         1.0       0.52      0.86      0.65     16759

    accuracy                           0.51     31965
   macro avg       0.49      0.50      0.43     31965
weighted avg       0.49      0.51      0.44     31965

